In [49]:
import pandas as pd

df = pd.read_csv("data/demand.csv", index_col="ID")

In [52]:
df = df.dropna()

In [53]:
df

,Store ID,Total Price,Base Price,Units Sold
ID,,,,
1,8091,99.0375,111.8625,20
2,8091,99.0375,99.0375,28
3,8091,133.9500,133.9500,19
4,8091,133.9500,133.9500,44
5,8091,141.0750,141.0750,52
...,...,...,...,...
212638,9984,235.8375,235.8375,38
212639,9984,235.8375,235.8375,30
212642,9984,357.6750,483.7875,31


In [54]:
target_sensor = "Units Sold"
features = list(df.columns.difference([target_sensor]))
features

['Base Price', 'Store ID', 'Total Price']

In [55]:
forecast_lead = 10
target = f"{target_sensor}_lead_{forecast_lead}"
target

'Units Sold_lead_10'

In [56]:
df[target] = df[target_sensor].shift(-forecast_lead)
df

C:\Users\manyu\AppData\Local\Temp\ipykernel_16888\4082843283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target] = df[target_sensor].shift(-forecast_lead)


,Store ID,Total Price,Base Price,Units Sold,Units Sold_lead_10
ID,,,,,
1,8091,99.0375,111.8625,20,120.0
2,8091,99.0375,99.0375,28,40.0
3,8091,133.9500,133.9500,19,68.0
4,8091,133.9500,133.9500,44,87.0
5,8091,141.0750,141.0750,52,186.0
...,...,...,...,...,...
212638,9984,235.8375,235.8375,38,NaN
212639,9984,235.8375,235.8375,30,NaN
212642,9984,357.6750,483.7875,31,NaN


In [17]:
df = df.iloc[:-forecast_lead]
df

,Average,Srinivaspur A,Srinivaspur B,Srinivaspur A_lead15
DateTime,,,,
2024-03-08 22:46:59,50.0,71.32,51.52,75.89
2024-03-08 22:48:59,NaN,71.89,52.04,78.98
2024-03-08 22:50:59,NaN,72.85,52.95,77.86
2024-03-08 22:52:59,NaN,73.19,54.47,77.50
2024-03-08 22:54:59,NaN,76.23,55.02,78.93
...,...,...,...,...
2024-03-10 22:05:56,NaN,38.28,31.57,39.09
2024-03-10 22:07:55,NaN,37.60,30.89,38.96
2024-03-10 22:09:55,NaN,37.44,31.42,39.15


In [57]:
test_start = df.shape[0]//2

df_train = df.iloc[:test_start].copy()
df_test = df.iloc[test_start:].copy()

print("Test set fraction:", len(df_test) / len(df))

Test set fraction: 0.500003330025508


In [58]:
df_train

,Store ID,Total Price,Base Price,Units Sold,Units Sold_lead_10
ID,,,,,
1,8091,99.0375,111.8625,20,120.0
2,8091,99.0375,99.0375,28,40.0
3,8091,133.9500,133.9500,19,68.0
4,8091,133.9500,133.9500,44,87.0
5,8091,141.0750,141.0750,52,186.0
...,...,...,...,...,...
106219,9984,240.8250,240.8250,16,36.0
106220,9984,109.7250,201.6375,59,27.0
106221,9984,227.2875,227.2875,27,33.0


In [59]:
df_test

,Store ID,Total Price,Base Price,Units Sold,Units Sold_lead_10
ID,,,,,
106226,9984,177.4125,213.0375,12,159.0
106227,8091,104.0250,104.0250,40,113.0
106228,8091,106.1625,106.1625,43,162.0
106229,8091,117.5625,132.5250,47,84.0
106230,8091,118.2750,133.9500,46,103.0
...,...,...,...,...,...
212638,9984,235.8375,235.8375,38,NaN
212639,9984,235.8375,235.8375,30,NaN
212642,9984,357.6750,483.7875,31,NaN


In [60]:
df.shape

(150149, 5)

In [31]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

In [32]:
target_mean

62.48769662921349

In [33]:
target_stdev

28.45741258149261

In [41]:
for c in df_train.columns:
    print(c)
    mean = df_train[c].mean()
    stdev = df_train[c].std()
    print(mean, stdev)

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

Average
50.0 nan
Srinivaspur A
63.07842696629213 28.43820516229189
Srinivaspur B
49.0554915730337 19.27905034062874
Srinivaspur A_lead15
62.48769662921349 28.45741258149261


In [42]:
import torch
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [43]:
target

'Srinivaspur A_lead15'

In [44]:
features

['Average', 'Srinivaspur B']

In [45]:
i = 27
sequence_length = 4

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)

X, y = train_dataset[i]

In [46]:
X

tensor([[   nan, 0.7171],
        [   nan, 0.8260],
        [   nan, 0.8530],
        [   nan, 0.8732]])

In [47]:
y

tensor(1.1882)